In [5]:
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from datetime import datetime
from icecream import ic

In [1]:
def log_progress(message):
    """This function log the mentioned message of a given stage of the
    code execution to a log file.funtion return nothing"""
    with open('./logs/code_log.txt','a') as f:
        f.write(f'{datetime.now()}:{message}\n')

In [15]:
def extract(url,table_attributes):
    """ This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. """
    
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    # print(soup)
    table = soup.find('span', string=table_attributes).find_next('table')
    # soup=BeautifulSoup(requests.get(url).text,'html.parser')
    # table=soup.find('span',string=table_attributes).find_next('table')
    df=pd.read_html(StringIO(str(table)))[0]
    
    log_progress('Data extraction.complete.Initiating Transformation process')
    
    return df

In [25]:
def transform(df,csv_path):
  
  """ This function accesses the CSV file for exchange rate
  information, and adds three columns to the data frame, each
  containing the transformed version of Market Cap column to
  respective currencies"""
     
  exchange_rate=pd.read_csv(csv_path,index_col=0).to_dict()['Rate']
    
  df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
  df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
  df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)
    
  (df['MC_EUR_Billion'][4])

  log_progress('Data transformation complete. Initiating Loading process')

  return df 

In [43]:
def load_to_csv(df,output_path):
    """ This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing."""
    print(output_path)
    df.to_csv(output_path)
    
    log_progress('data saved to csv file')

In [34]:
def load_to_db(df, sql_connection, table_name):
    """ This function saves the final data frame to a database
    table with the provided name. Function returns nothing."""

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

    log_progress('Data loaded to Database as a table, Executing queries')

In [53]:
if __name__ == "__main__":
    url='https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output='./output/lagest_Bank_Data.csv'
    database='./output/Bank.db'
    table_name='Largest_bank'
    
    log_progress('Premilinar complete.Initiating ETL Process')
    
    df=extract(url,'By market capitalization')
     # print(df)
    
    df = transform(df,'./input/exchange_rate.csv')
    # print(df)
    load_to_csv(df,output)
    
    def run_querry(querry_statements , sql_connection):
        """This function runs the querry on the database table
        and prints the output on the terminal.Function require nothing."""
        
        cursor = sql_connection.cursor()
        cursor.execute(querry_statements)
        result = cursor.fetchall()
        
        #for now in result:
        # ic(row)
        
        log_progress('process Complete')
        return result
    
    with sqlite3.connect(database) as conn:
        load_to_db(df, conn, table_name)

        print(run_querry('SELECT * FROM Largest_bank', conn))

        print(run_querry('SELECT AVG(MC_GBP_Billion) FROM Largest_bank', conn))

        print(run_querry('SELECT "Bank name" FROM Largest_bank LIMIT 5', conn))

./output/lagest_Bank_Data.csv
[(1, 'JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71), (2, 'Bank of America', 231.52, 185.22, 215.31, 19204.58), (3, 'Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75), (4, 'Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41), (5, 'HDFC Bank', 157.91, 126.33, 146.86, 13098.63), (6, 'Wells Fargo', 155.87, 124.7, 144.96, 12929.42), (7, 'HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26), (8, 'Morgan Stanley', 140.83, 112.66, 130.97, 11681.85), (9, 'China Construction Bank', 139.82, 111.86, 130.03, 11598.07), (10, 'Bank of China', 136.81, 109.45, 127.23, 11348.39)]
[(151.98700000000002,)]
[('JPMorgan Chase',), ('Bank of America',), ('Industrial and Commercial Bank of China',), ('Agricultural Bank of China',), ('HDFC Bank',)]
